In [1]:
import sys
sys.path.append("..")

In [2]:
from db.duckdb.duckdbhelper import DuckDBDatabaseHelper
from pprint import pprint
import pandas as pd

In [3]:
NUM_ROWS = 1500000

In [4]:
db = DuckDBDatabaseHelper("../meters.db")

In [8]:
sql_query ="""

SELECT day,energy_sum, 
temperatureMin, 
temperatureMax,
uvIndex,
dewPoint,                  
Type,
date_part('month', day) as month,
m.LCLid
FROM  meters m LEFT JOIN info_household ih
ON m.LCLid = ih.LCLid
LEFT JOIN weather_daily_darksky_modified wd
ON m.day = wd.formatted_temperatureMaxTime_weather_daily_darksky
LEFT JOIN uk_bank_holidays uk_bh
ON m.day = uk_bh."Bank holidays"

WHERE date_part('year', day) = 2013

"""

db.connect()
records = db.fetch_all(sql_query)
db.close_connection()

Connected to DuckDB database: ../meters.db
Fetched 1966727 rows.
Connection closed.


In [9]:
records[:1]

[(datetime.date(2013, 1, 3),
  10.074,
  9.65,
  11.41,
  1.0,
  9.12,
  None,
  1,
  'MAC000002')]

In [11]:
COLS = [
"day","energy_sum", 
"temperatureMin", 
"temperatureMax",
"uvIndex",
"dewPoint",                  
"Type","month","LCLid" ]

In [12]:
df_model = pd.DataFrame(records,columns = COLS)

In [13]:
df_model.head()

,day,energy_sum,temperatureMin,temperatureMax,uvIndex,dewPoint,Type,month,LCLid
0,2013-01-03,10.074,9.65,11.41,1.0,9.12,None,1,MAC000002
1,2013-01-06,10.293,6.91,8.61,1.0,7.00,None,1,MAC000002
2,2013-01-07,9.439,7.10,9.08,1.0,6.78,None,1,MAC000002
3,2013-01-11,10.979,0.89,5.70,1.0,1.68,None,1,MAC000002
4,2013-01-12,10.586,1.75,4.53,1.0,0.79,None,1,MAC000002


In [14]:
df_model2 = df_model[df_model["LCLid"] == 'MAC000002']

In [15]:
df_model2['Type_Binary'] = df_model2['Type'].apply(lambda x: 1 if x is not None else 0)

C:\Users\Ambarish Ganguly\AppData\Local\Temp\ipykernel_7720\3940123378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model2['Type_Binary'] = df_model2['Type'].apply(lambda x: 1 if x is not None else 0)


In [17]:
len(df_model2)

370

In [18]:
df_model2.columns

Index(['day', 'energy_sum', 'temperatureMin', 'temperatureMax', 'uvIndex',
       'dewPoint', 'Type', 'month', 'LCLid', 'Type_Binary'],
      dtype='object')

In [19]:
df_model2.to_csv("../model_data/train_day.csv",index = False)